In [66]:
import numpy as np
import math as math
import numpy.linalg as LA

# Funckija koja racuna teziste skupa tacaka u 2D
def izracunaj_teziste(tacke):
  n = len(tacke)
  C_x = 0
  C_y = 0

  for tacka in tacke:
    C_x += tacka[0]
    C_y += tacka[1]

  return (C_x/n, C_y/n)

def izracunaj_prosek_rastojanja(tacke):
  rastojanje = 0
  n = len(tacke)
  for tacka in tacke:
    rastojanje += math.sqrt(tacka[0]**2 + tacka[1]**2)

  return rastojanje/n    
'''
 Funckija koja vrsi normalizaciju tako sto prvo svaku tacku u skupu translira za vektor CO
 gde je C teziste skupa tacaka, a O koordinatni pocetak. Zatim se vrsi skaliranje kooridnata
 tako da prosecno rastojanje skupa tacaka do koordinatnog pocetka bude sqrt(2). Ovako dobijen
 skup tacaka se naziva normalizovan.
'''
def normalizuj(tacke):

  (C_x, C_y) = izracunaj_teziste(tacke)

  nova_lista = []
  for tacka in tacke:
    x = tacka[0] - C_x
    y = tacka[1] - C_y
    nova_lista.append([x, y])

  lam = izracunaj_prosek_rastojanja(nova_lista)
  tacke = []

  for tacka in nova_lista:
    x = (tacka[0]/lam)*math.sqrt(2)
    y = (tacka[1]/lam)*math.sqrt(2)
    tacke.append([x, y])

  matrica_translacije = np.array([[1, 0, -C_x], [0, 1, -C_y], [0, 0, 1]])
  matrica_homotetije = np.array([[math.sqrt(2)/lam, 0, 0], [0, math.sqrt(2)/lam, 0], [0, 0, 1]])
  matrica_transformacije = matrica_homotetije @ matrica_translacije

  return tacke, matrica_transformacije

tacke = [[1, 1], [5, 2], [6, 4], [-1, 7]]
tacke, matrica = normalizuj(tacke)
print(tacke)
print(matrica)


[[-0.698445552349397, -0.9977793604991386], [0.8980014244492246, -0.5986676162994831], [1.2971131686488802, 0.19955587209982772], [-1.4966690407487078, 1.396891104698794]]
[[ 0.39911174  0.         -1.0975573 ]
 [ 0.          0.39911174 -1.3968911 ]
 [ 0.          0.          1.        ]]


In [67]:
def prebaci_u_projektivne(tacke):
  lista = []
  for tacka in tacke:
    x = tacka[0]
    y = tacka[1]
    lista.append([x, y, 1])
  return lista

def prebaci_u_afine(tacke):
  lista = []
  for tacka in tacke:
    x = tacka[0]/tacka[2]
    y = tacka[1]/tacka[2]
    lista.append([x, y])
  return lista

# Funkcija koja formira matricu cije su kolone redom vektori A, B i C

def formiraj_matricu(A, B, C):
  array = np.array([A, B, C]).T
  return array

def izvuci_kolone(matrica):
  prva_kolona = []
  for i in range(3):
    prva_kolona.append(matrica[i][0])
  druga_kolona = []
  for i in range(3):
    druga_kolona.append(matrica[i][1])
  treca_kolona = []
  for i in range(3):
    treca_kolona.append(matrica[i][2])

  return prva_kolona, druga_kolona, treca_kolona

'''
  Ova funkcija predstavlja implementaciju naivnog algoritma za odredjivanje 
  matrice projektivnog preslikavanja za otklanjanje projektivne distorzije.
'''

def naivni(originali, slike):
  if (len(originali) != 4 or len(slike) != 4 or len(originali) != len(slike)):
    print("Funckija prima tacno 4 tacke!")
    return None

  originali = prebaci_u_projektivne(originali)
  slike = prebaci_u_projektivne(slike)
  
  A1, B1, C1, D1 = originali[0], originali[1], originali[2], originali[3]
  A2, B2, C2, D2 = slike[0], slike[1], slike[2], slike[3]
  D1 = np.array([D1]).T
  D2 = np.array([D2]).T

  matrica_1 = formiraj_matricu(A1, B1, C1)
  matrica_2 = formiraj_matricu(A2, B2, C2)
  
  # Potrbno je pronaci alfa, beta i gama iz izraza
  # D1 = alfa*A1 + beta*B1 + gama*C1
  # Slicno vazi i za kolone matrice matrica_2
  
  X = np.linalg.inv(matrica_1).dot(D1)
  Y = np.linalg.inv(matrica_2).dot(D2)

  prva_kolona, druga_kolona, treca_kolona = izvuci_kolone(matrica_1)
  P1 = formiraj_matricu(X[0]*prva_kolona, X[1]*druga_kolona, X[2]*treca_kolona)

  prva_kolona, druga_kolona, treca_kolona = izvuci_kolone(matrica_2)
  P2 = formiraj_matricu(Y[0]*prva_kolona, Y[1]*druga_kolona, Y[2]*treca_kolona)

  P = np.matmul(P2, LA.inv(P1))
  return P

print(naivni([[1, 1], [5, 2], [6, 4], [-1, 7]], [[0, 0], [10, 0], [10, 5], [0, 5]]))

[[ 0.44871795  0.14957265 -0.5982906 ]
 [-0.19230769  0.76923077 -0.57692308]
 [-0.07277526  0.07378079  0.41075918]]


In [23]:
'''
  Ova funkcija predstavlja implementaciju DLT algoritma za odredjivanje matrice
  projektivnog preslikavanja radi otklanjanja projektivne distorzije
'''

def DLT(originali, slike):
  if(len(originali) < 4 or len(slike) < 4 or len(originali) != len(slike)):
    print("Losi argumenti funkcije!")
    return None

  n = len(originali)

  originali = prebaci_u_projektivne(originali)
  slike = prebaci_u_projektivne(slike)

  matrica = np.array([]).reshape(0, 9)

  for i in range(0, n):
    tacka1 = originali[i]
    tacka2 = slike[i]

    pomocna = np.array([[0, 0, 0, -tacka2[2]*tacka1[0], -tacka2[2]*tacka1[1], -tacka2[2]*tacka1[2], tacka2[1]*tacka1[0], tacka2[1]*tacka1[1], tacka2[1]*tacka1[2]], 
                              [tacka2[2]*tacka1[0], tacka2[2]*tacka1[1], tacka2[2]*tacka1[2], 0, 0, 0, -tacka2[0]*tacka1[0], -tacka2[0]*tacka1[1], -tacka2[0]*tacka1[2]]])

    matrica = np.concatenate((matrica, pomocna), axis=0)

  _, _, v = np.linalg.svd(matrica, full_matrices=True)

  v = v[-1]
  return np.array([[v[0], v[1], v[2]], [v[3], v[4], v[5]], [v[6], v[7], v[8]]])

print(naivni([[1, 1], [5, 2], [6, 4], [-1, 7]], [[0, 0], [10, 0], [10, 5], [0, 5]]))
print('-------------------------------------------------------------')
DLT([[1, 1], [5, 2], [6, 4], [-1, 7]], [[0, 0], [10, 0], [10, 5], [0, 5]])



[[ 0.44871795  0.14957265 -0.5982906 ]
 [-0.19230769  0.76923077 -0.57692308]
 [-0.07277526  0.07378079  0.41075918]]
-------------------------------------------------------------


array([[ 0.3418787 ,  0.11395957, -0.45583826],
       [-0.14651944,  0.58607777, -0.43955833],
       [-0.05544755,  0.05621367,  0.31295787]])

In [61]:
'''
  Ova funkcija predstavlja implementaciju unapredjenog DLT algoritma za odredjivanje 
  matrice projektivnog preslikavanja radi otklanjanja projektivne distorzije
'''

def DLT_unapredjeni(originali, slike):
  originali, matrica1 = normalizuj(originali)
  slike, matrica2 = normalizuj(slike)

  matrica = DLT(originali, slike)

  return np.linalg.inv(matrica2) @ matrica @ matrica1

print(DLT_unapredjeni([[1, 1], [5, 2], [6, 4], [-1, 7]], [[0, 0], [10, 0], [10, 5], [0, 5]]))
print()
print(DLT_unapredjeni([[3, 2], [4, -2], [6, -3], [9, 4]], [[5, -2], [15, 8], [10, 13], [0, 3]]))

[[ 0.49851447  0.16617149 -0.66468596]
 [-0.21364906  0.85459623 -0.64094717]
 [-0.08085151  0.08196863  0.45634321]]

[[-0.27858161 -0.30790599  3.7388585 ]
 [ 0.85683047 -0.44656842 -2.59227523]
 [ 0.08196863  0.08085151  0.04985145]]


In [68]:
'''
  U ovom delu koda uporedicemo rad DLT i unapredjenog DLT algoritma i pokazati
  da je unapredjeni DLT algoritam otporan na promenu koordinata
'''

originali = np.array([[1, 1], [5, 2], [6, 4], [-1, 7]])
slike = np.array([[0, 0], [10, 0], [10, 5], [0, 5]])

originali_proj = np.array(prebaci_u_projektivne(originali))
slike_proj = np.array(prebaci_u_projektivne(slike))

matricaDLT = DLT(originali, slike)
matricaDLTUn = DLT_unapredjeni(originali, slike)

matricaDLT = matricaDLT / (matricaDLTUn[0][0] / matricaDLT[0][0])
matricaDLTUn = matricaDLTUn / (matricaDLTUn[0][0] / matricaDLT[0][0])

print("Matrica transformacije dobijena DLT algoritmom:")
print(matricaDLT)
print()
print("Matrica transformacije dobijena unapredjenim DLT algoritmom:")
print(matricaDLTUn)
print()

transf_orig = np.array([[0, 1, 2], [-1, 0, 3], [0, 0, 1]])
transf_slik = np.array([[1, -1, 5], [1, 1, -2], [0, 0, 1]])

originali = np.array([*map(lambda x: transf_orig @ x.T, originali_proj)])
slike = np.array([*map(lambda x: transf_slik @ x.T, slike_proj)])

originali = prebaci_u_afine(originali)
slike = prebaci_u_afine(slike)

matrica = np.array(DLT_unapredjeni(originali, slike))
print("Matrica dobijenja unapredjenim DLT algoritmom sa izmenjenim koordinatama:")
print(matrica)
print()
print("Matrica dobijena kada se vratimo u stare koordinate:")
print(LA.inv(transf_slik)@matrica @ transf_orig)


Matrica transformacije dobijena DLT algoritmom:
[[ 0.23445868  0.07815289 -0.31261157]
 [-0.10048229  0.40192917 -0.30144687]
 [-0.03802565  0.03855105  0.21462492]]

Matrica transformacije dobijena unapredjenim DLT algoritmom:
[[ 0.23445868  0.07815289 -0.31261157]
 [-0.10048229  0.40192917 -0.30144687]
 [-0.03802565  0.03855105  0.21462492]]

Matrica dobijenja unapredjenim DLT algoritmom sa izmenjenim koordinatama:
[[-0.27858161 -0.30790599  3.7388585 ]
 [ 0.85683047 -0.44656842 -2.59227523]
 [ 0.08196863  0.08085151  0.04985145]]

Matrica dobijena kada se vratimo u stare koordinate:
[[ 0.49851447  0.16617149 -0.66468596]
 [-0.21364906  0.85459623 -0.64094717]
 [-0.08085151  0.08196863  0.45634321]]
